# Install required libraries

In [4]:
!pip install transformers sentencepiece datasets evaluate sacrebleu bert-score --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.9 MB/s eta 0:00:00


# Import required libraries

In [5]:
import pandas as pd
import numpy as np
from datasets import Dataset, load_dataset
from transformers import (
    MT5ForConditionalGeneration,
    MT5Tokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
import evaluate
import torch

# Upload dataset

In [18]:
from google.colab import files
uploaded = files.upload()

# Extract uploaded ZIP file

In [1]:
import zipfile
import io

# Replace with the actual uploaded file name
file_name = "Output.zip"

with zipfile.ZipFile(file_name, 'r') as zip_ref:
    zip_ref.extractall("/content")
print("Files extracted successfully!")

Files extracted successfully!


# Load Dataset from CSV Files

In [8]:
# Upload the CSVs if not already in Colab, or adjust paths accordingly
train_path = "/content/Output/train_hi.csv"  # <-- Update if path is different
val_path = "/content/Output/val_hi.csv"

# Load using HuggingFace Datasets
dataset = load_dataset("csv", data_files={"train": train_path, "validation": val_path})

# Show a sample
print(dataset["train"][0])

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

{'id': 'dlg_cbb0155f815386275f372989ec609d9f', 'source': 'train.csv', 'type': 'dialogue', 'prompt': 'क्या इस बारे में मुझे कुछ चिंतित होना चाहिए? क्या मुझे इसके बारे में कुछ करना चाहिए?', 'response': 'उत्तर इस बात पर निर्भर करता है कि आपके बेटे के जीवन के अन्य क्षेत्र कैसा कर रहे हैं। क्या वह खुश है या वह खुश दिखता है, अवकाश के दौरान अकेले खेल रहा है? क्या अवकाश के दौरान वह जिन छात्रों के साथ है, उनके अलावा अन्य सामाजिक समूहों में उसके दोस्त हैं? उसकी शैक्षणिक प्रगति कैसी है? उसके सहपाठियों के बीच उसका सामाजिक एकीकरण कैसा है? क्या घर और परिवार के माहौल में कोई विशेष या असामान्य परिस्थितियाँ हैं? इस सूची के माध्यम से एक पूर्ण विचार बनाएं कि क्या आपका बेटा केवल अकेले समय पसंद करता है और अवकाश के दौरान इस विकल्प को लेता है, या यदि कोई हो तो उपरोक्त विकल्प अपनाता है। क्षेत्र उसके लिए तनाव या कठिनाई दर्शाते हैं और जिन्हें आगे समझने और संभालने की आवश्यकता है।', 'lang': 'hi', 'orig_id': None, 'fp': '066312c81016ef51cd5a0c80bbeb84ec'}


# Initialize Tokenizer and Model

In [9]:
model_name = "google/mt5-small"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.mt5.tokenization_mt5.MT5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Preprocess the Dataset

In [10]:
max_input_length = 128
max_target_length = 128

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["prompt"], max_length=max_input_length, truncation=True, padding="max_length"
    )
    labels = tokenizer(
        examples["response"], max_length=max_target_length, truncation=True, padding="max_length"
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/1140 [00:00<?, ? examples/s]

Map:   0%|          | 0/229 [00:00<?, ? examples/s]

# Set Training Arguments

In [11]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results_hindi_chatbot",
    eval_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=100,
    push_to_hub=False,
    report_to="none"
)

# Initialize Trainer and Start Training

In [12]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    processing_class=tokenizer,
)

trainer.train()


Epoch,Training Loss,Validation Loss
1,3.588900,2.493386
2,2.891300,2.339172
3,2.752000,2.297786


TrainOutput(global_step=855, training_loss=3.467729231628061, metrics={'train_runtime': 432.0349, 'train_samples_per_second': 7.916, 'train_steps_per_second': 1.979, 'total_flos': 452080985702400.0, 'train_loss': 3.467729231628061, 'epoch': 3.0})

# Save the Trained Model

In [13]:
trainer.save_model("./hindi_chatbot_mt5")
tokenizer.save_pretrained("./hindi_chatbot_mt5")

('./hindi_chatbot_mt5/tokenizer_config.json',
 './hindi_chatbot_mt5/special_tokens_map.json',
 './hindi_chatbot_mt5/spiece.model',
 './hindi_chatbot_mt5/added_tokens.json')

# Setup Device and Define Chat Function

In [14]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

def chat_with_bot(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=128).to(device)
    output = model.generate(**inputs, max_length=80, num_beams=4, early_stopping=True)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Enhanced Chatbot Generation Function

In [15]:
def chat_with_bot(text):
    prompt = f"User: {text}\nBot:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=128).to(device)
    output = model.generate(
        **inputs,
        max_length=100,
        num_beams=5,
        do_sample=True,
        top_p=0.9,
        top_k=40,
        repetition_penalty=1.5,   # 👈 prevents loops
        early_stopping=True
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Punctuation Restoration

In [16]:
!pip install deepmultilingualpunctuation
from deepmultilingualpunctuation import PunctuationModel

punc_model = PunctuationModel()

def restore_punctuation(text):
    return punc_model.restore_punctuation(text)


config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/406 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/token_classification.py:186: UserWarning: `grouped_entities` is deprecated and will be removed in version v5.0.0, defaulted to `aggregation_strategy="AggregationStrategy.NONE"` instead.
  warnings.warn(


# Mental-Health–Tuned Chatbot Response Function

In [17]:
def chat_with_bot(text):
    prompt = f"""
आप एक बहुत ही दयालु और शांत मानसिक स्वास्थ्य सहायक हैं।
आप इंसान की तरह सरल, सहज और दिल से बात करते हैं।
आपके जवाब हमेशा छोटे, साफ और बहुत कोमल होने चाहिए (2–3 वाक्य)।
कठिन या औपचारिक शब्द (जैसे परामर्शदाता, व्यवहार, प्रक्रिया) का उपयोग बिल्कुल न करें।

आपका लक्ष्य:
- व्यक्ति की बात को ध्यान से सुनना
- उसकी भावना को समझना
- उसे थोड़ा आराम और भरोसा दिलाना
- बिल्कुल सरल हिंदी में जवाब देना

उदाहरण:
User: मैं बहुत तनाव में हूँ।
Bot: यह सुनकर दुख हुआ। कभी-कभी तनाव बहुत भारी लग सकता है। क्या आप बता सकते हैं कि सबसे ज़्यादा क्या परेशान कर रहा है?

User: मैं बहुत उदास हूँ।
Bot: मैं समझ सकता हूँ कि यह कितना मुश्किल होता है। आप अकेले नहीं हैं। अगर चाहें तो बताइए, यह उदासी किस वजह से आ रही है?

अब बातचीत जारी रखें:

User: {text}
Bot:
"""

    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=620).to(model.device)

    output = model.generate(
        **inputs,
        max_new_tokens=90,
        do_sample=True,
        temperature=0.7,       # 👈 less random, more stable emotional tone
        top_p=0.9,
        top_k=30,              # 👈 makes output more natural
        repetition_penalty=1.15,
        no_repeat_ngram_size=2
    )

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    response = response.split("Bot:")[-1].strip()
    return response


In [18]:
unsafe_words = ["आत्महत्या", "खुदकुशी", "मरना", "मरे", "खत्म", "suicide", "kill"]

def safe_chat(text):
    # Emergency emotion detection
    if any(w in text.lower() for w in unsafe_words):
        return (
            "मुझे बहुत दुख है कि आप इतनी गहरी तकलीफ़ महसूस कर रहे हैं। "
            "आप अकेले नहीं हैं। किसी भरोसेमंद दोस्त, परिवार के सदस्य या "
            "मानसिक स्वास्थ्य विशेषज्ञ से तुरंत बात करना बहुत मददगार हो सकता है। "
            "अगर आपको तुरंत मदद की ज़रूरत हो, तो अपने नज़दीकी हेल्पलाइन से संपर्क करें।"
        )

    return chat_with_bot(text)


In [19]:
chat_with_bot("	क्या इस बारे में मुझे कुछ चिंतित होना चाहिए? क्या मुझे इसके बारे में कुछ करना चाहिए?")


'आपके पास एक ही अच्छी समस्या है। कई लोगों के बारे में अधिक जानकारी नहीं हैं, क्योंकि हमेशा आपको अपने साथ बात करते हैं। इससे यह सुनकर बहुत अधिक महत्वपूर्ण है कि जब आपका विचार उस व्यक्ति को इसे बताना होता है, तो यदि आप अपनी भावनाओं को सु'

In [20]:
chat_with_bot("मुझे बहुत तनाव हो रहा है।")

'क्या आपको यह बताना होगा कि आप अपने विचारों को संभालना चाहते हैं? आप इस प्रकार के बारे में बात कर रहे हैं। कुछ लोगों के दौरान आगे बढ़ता है और आपके साथ काम करने का एक समय हो सकता हैं, तो ऐसा लगता हूं कि एक चिकित्सक से जुड़ा हुआ है। आप'

# Test the Chatbot with Sample Inputs

# Hindi Chatbot Evaluation

In [21]:
# === Hindi Chatbot Evaluation (for columns: prompt, response) ===
!pip install -q evaluate sacrebleu rouge_score bert_score transformers>=4.30.0

import torch
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
import evaluate
import pandas as pd
from tqdm import tqdm

# ---------- CONFIG ----------
model_path = "/content/hindi_chatbot_mt5"    # or your model folder
test_csv = "/content/Output/test_hi.csv"     # test file
device = "cuda" if torch.cuda.is_available() else "cpu"
max_input_length = 256
max_gen_length = 128
batch_size = 8
# ----------------------------

# Load tokenizer & model
tokenizer = MT5Tokenizer.from_pretrained(model_path)
model = MT5ForConditionalGeneration.from_pretrained(model_path).to(device)
model.eval()

# Load dataset
df = pd.read_csv(test_csv)
inputs_list = df["prompt"].astype(str).tolist()
refs_list = df["response"].astype(str).tolist()

# Generate predictions
predictions = []
for i in tqdm(range(0, len(inputs_list), batch_size), desc="Generating"):
    batch = inputs_list[i : i + batch_size]
    enc = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_input_length).to(device)
    with torch.no_grad():
        outputs = model.generate(
            **enc,
            max_new_tokens=max_gen_length,
            num_beams=4,
            no_repeat_ngram_size=3,
            repetition_penalty=2.0,
            early_stopping=True
        )
    preds = [tokenizer.decode(o, skip_special_tokens=True, clean_up_tokenization_spaces=True) for o in outputs]
    predictions.extend(preds)

# Evaluate using BLEU, ROUGE, BERTScore
bleu = evaluate.load("sacrebleu")
rouge = evaluate.load("rouge")
bertscore = evaluate.load("bertscore")

bleu_res = bleu.compute(predictions=predictions, references=[[r] for r in refs_list])
rouge_res = rouge.compute(predictions=predictions, references=refs_list)
bertscore_res = bertscore.compute(predictions=predictions, references=refs_list, lang="hi")

print("\n=== 📊 Evaluation Results ===")
print(f"BLEU Score       : {bleu_res['score']:.3f}")
print(f"ROUGE-1 F-measure: {rouge_res['rouge1']:.3f}")
print(f"ROUGE-L F-measure: {rouge_res['rougeL']:.3f}")
print(f"BERTScore (F1)   : {sum(bertscore_res['f1'])/len(bertscore_res['f1']):.4f}")

# Save detailed outputs
out_df = pd.DataFrame({
    "prompt": inputs_list,
    "reference": refs_list,
    "prediction": predictions
})
out_df.to_csv("/content/hindi_eval_results.csv", index=False)
print("\n✅ Saved detailed results to /content/hindi_eval_results.csv")


Generating: 100%|██████████| 20/20 [01:04<00:00,  3.21s/it]


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]


=== 📊 Evaluation Results ===
BLEU Score       : 1.656
ROUGE-1 F-measure: 0.000
ROUGE-L F-measure: 0.000
BERTScore (F1)   : 0.7067

✅ Saved detailed results to /content/hindi_eval_results.csv


In [22]:
print(chat_with_bot("आज मौसम कैसा है?"))

क्या आपका यह अनुभव करने के लिए एक सरल बात है? क्या ऐसा लगता है कि आपको कुछ चिंता नहीं होगी? इसे आश्चर्य देना चाहिए? आपके बारे में सोचने की आवश्यकता है और वह हमारे बीच अधिक व्यापक संबंधों को अपनी भावनाओं से मदद करते हैं। क्या
